

# Events organization

Events: $E_1$, $E_2$, $E_3$, $E_4$ and $E_5$

People: $P_1$, $P_2$, $P_3$ and $P_4$.


Preferences:
- $P_1$ want to go to $E_1$ and $E_4$.
- $P_2$ want to go to $E_2$ and $E_3$.
- $P_3$ want to go to $E_4$ and $E_5$.
- $P_4$ want to go to $E_3$ and $E_4$.

Binary variables:

- $x_i = 0$ if the i-th event is on Saturday
- $x_i = 1$ if the i-th event is on Sunday

Formulation:

$f(x_1,x_2,x_3,x_4,x_5) = 4 + x_1 + x_2 + 2x_3 + 3x_4 + x_5 -2x_1x_4-2x_2x_3-2x_4x_5-2x_3x_4$

Change of variable:

$x_i = \frac{1-z_i}{2}$

- $z_i = 1$ if the i-th event is on Saturday
- $z_i = -1$ if the i-th event is on Sunday

New formulation:

$f(z_1,z_2,z_3,z_4,z_5) = 6 - \frac{1}{2}z_1z_4- \frac{1}{2}z_2z_3- \frac{1}{2}z_4z_5- \frac{1}{2}z_3z_4$

In [3]:
import pennylane as qml
from pennylane import numpy as np

H = - 6 * qml.Identity(1) + \
    0.5 * qml.PauliZ(1) @ qml.PauliZ(4) + \
    0.5 * qml.PauliZ(2) @ qml.PauliZ(3) + \
    0.5 * qml.PauliZ(4) @ qml.PauliZ(5) + \
    0.5 * qml.PauliZ(3) @ qml.PauliZ(4) 
    
print(H)

  (-6.0) [I1]
+ (0.5) [Z1 Z4]
+ (0.5) [Z2 Z3]
+ (0.5) [Z4 Z5]
+ (0.5) [Z3 Z4]


In [4]:
dev = qml.device("default.qubit", wires = H.wires)

@qml.qnode(dev)
def circuit(params):
    
    for param, wire in zip(params, H.wires):
        qml.RY(param, wires = wire)
        
    return qml.expval(H)

circuit([0,0,0,0,0])

tensor(-4., requires_grad=True)

In [5]:
params = np.random.rand(len(H.wires))
opt = qml.AdagradOptimizer(stepsize = 0.5)
epochs = 200

for epoch in range(epochs):
    params = opt.step(circuit, params)
    
circuit(params)

tensor(-8., requires_grad=True)

In [6]:
dev = qml.device("default.qubit", wires = H.wires, shots = 1)

@qml.qnode(dev)
def circuit(params):
    
    for param, wire in zip(params, H.wires):
        qml.RY(param, wires = wire)
        
    return qml.sample()

circuit(params)

tensor([0, 1, 0, 1, 0], requires_grad=True)